In [321]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import time
import pandas as pd
import json
import requests

In [322]:
def get_authorization_code():
    auth_url = (
        f"https://login.microsoftonline.com/consumers/oauth2/v2.0/authorize?"
        f"client_id={client_id}&response_type=code&redirect_uri=http://localhost:5001/callback&scope=https://graph.microsoft.com/Mail.Send"
    )
    
    webbrowser.open(auth_url)
    
    return input("Enter the authorization code:")

In [323]:
def get_access_token(client_id, client_secret, tenant_id, auth_code, redirect_uri):
    url = "https://login.microsoftonline.com/consumers/oauth2/v2.0/token"
    
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    
    body = {
        'grant_type': 'authorization_code',
        'client_id': client_id,
        'client_secret': client_secret,
        'code': auth_code,
        'redirect_uri': redirect_uri,
        'scope': 'https://graph.microsoft.com/Mail.Send'
    }

    response = requests.post(url, headers=headers, data=body)
    
    if response.status_code == 200:
        return response.json().get('access_token')
    else:
        raise Exception(f"Failed to obtain access token: {response.text}")

In [324]:
import base64
import json
import requests


def encode_file_as_base64(file_path):
    with open(file_path, 'rb') as f:
        encoded_file = base64.b64encode(f.read()).decode('utf-8')
    return encoded_file


def prepare_attachment(file_path, attachment_name):
    encoded_content = encode_file_as_base64(file_path)
    
    return {
        "@odata.type": "#microsoft.graph.fileAttachment",
        "name": attachment_name,
        "contentBytes": encoded_content,
        "contentType": "application/pdf" 
    }

In [325]:
# Function to send email via Microsoft Graph API with attachments
def send_email_via_graph_api(access_token, recipient_email, subject, body, attachments=[]):
    url = f'https://graph.microsoft.com/v1.0/users/harish.thangaraj03@outlook.com/sendMail'
    
    # Create the email payload
    email_data = {
        "message": {
            "subject": subject,
            "body": {
                "contentType": "Text",
                "content": body
            },
            "toRecipients": [
                {
                    "emailAddress": {
                        "address": recipient_email
                    }
                }
            ],
            "attachments": attachments
        },
        "saveToSentItems": "true"
    }
    
    # Set the headers
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    
    # Send the email
    response = requests.post(url, headers=headers, data=json.dumps(email_data))
    
    if response.status_code == 202:
        print(f"Email successfully sent to {recipient_email}")
    else:
        print(f"Error sending email: {response.status_code} - {response.text}")

In [326]:
def personalize_email(prof_Lname, research_lab, pub1, pub2, application_reference, university):
    subject = f"Inquiry: Prospective Master's Student – {research_lab}"
    body = f"""
Dear Professor {prof_Lname},

I am a prospective master’s student in Computer Science at {university} for the Fall 2025 intake. I am currently an undergraduate researcher with a focus on Natural Language Processing, Computer Vision, and Machine Learning with research experience at Vellore Institute of Technology, University of Pretoria, and industrial R&D at several startup organizations.

I came across your profile on {university}'s research page and was particularly intrigued by your publication ‘{pub1}’ which led me to your work ‘{pub2}’. I am confident that my research interests align with your expertise, and I believe I can contribute meaningfully maintaining the high standards of work you expect.

I am looking forward to discussing potential opportunities at your {research_lab} Laboratory during my graduate studies. I would also appreciate any suggestions or referrals if you cannot accommodate me.

Attached with this mail:
1. Academic CV
2. Research Statement
3. Application reference number: {application_reference}

Thank you, professor, for your patience and time here.

Your Sincerely,
Harish Thangaraj


    """
    return subject, body


In [327]:
import pandas as pd
import requests
import io

sheet_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vR-hE5z3EtaFbmkr2oK26-Zfep-5QvQDyY31rsAwsMbbkL9c7ATebzT6ft1Q83pPYtQRYUFJ3Qx6oXW/pub?gid=0&single=true&output=csv"

# Fetch the CSV content with SSL verification disabled
response = requests.get(sheet_url, verify=False)

# Read the content into a pandas DataFrame
df = pd.read_csv(io.StringIO(response.text))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'docs.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'doc-10-0k-sheets.googleusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [328]:
def update_google_sheet(df):
    # Set up the Google Sheets API client
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
    client = gspread.authorize(creds)
    
    # Open the Google Sheet by name or URL
    sheet = client.open("Your Google Sheet Name").sheet1  # Use the correct name
    
    # Update the sheet with the DataFrame values
    # Clear the existing data
    sheet.clear()
    
    # Set the DataFrame to the sheet
    sheet.update([df.columns.values.tolist()] + df.values.tolist())

In [330]:
def send_cold_emails(access_token):
    # Sender's credentials
    sender_email = 'harish.thangaraj03@outlook.com'
    sender_password = '' #app password
    
    
    pending_list=df[df['Sent'] == 'No']
    
    # List of professors to send emails to
    professors = pending_list.to_dict(orient='records')

    # Loop through the list of professors and send personalized emails
    for prof in professors:
        subject, body = personalize_email(prof['Professor Name'].split()[-1], 
                                          prof['Research Lab'], 
                                          prof['Publication 1'], 
                                          prof['Publication 2'], 
                                          prof['Application reference'],
                                          prof['University'])
        
        attachments = [
        prepare_attachment('CV_Harish.pdf', 'Harish_CV.pdf'),
        prepare_attachment('Research_Statement.pdf', 'Research_Statement.pdf')]
        
        send_email_via_graph_api(access_token, prof['Email'], subject, body, attachments)
        
        df.loc[df['Email'] == prof['Email'], 'Sent'] = 'Yes'
        
        # Pause to avoid being flagged as spam
        time.sleep(60)  # Wait 1 minute between each email


In [337]:
if __name__ == "__main__":
    client_id = '0d5565e0-0242-46df-a473-88dbe84dd59e'
    client_secret = ''
    tenant_id = 'fd4d4a94-80e6-4ee7-832a-cd70b6a6ab7d'
    sender_email = 'harish.thangaraj03@outlook.com'
    
    print("Please verify the data")
    print(df)
    
    verify=int(input("Enter 1 if okay to proceed else enter 0"))

    if (verify == 1):
        auth_code=get_authorization_code()
        access_token=get_access_token(client_id, client_secret, tenant_id, auth_code,'http://localhost:5001/callback')
        send_cold_emails(access_token)

Please verify the data
     Professor Name      Research Lab           Publication 1  \
0  Harish Thangaraj               NLP  Cross lingual transfer   
1  Thangaraj Harish   Computer vision  Cross lingual transfer   
2   Aishwariya Iyer   Computer vision               Hand sign   
3     Diya Katariya               NLP     Color sorting robot   
4      Eshaan Joshi  Machine Learning     Color sorting robot   

                Publication 2  Application reference    University  \
0  BERT based academic system               12348473          UCSD   
1  BERT based academic system              909090909         Umass   
2              Stock analysis               34589482      San jose   
3      Customer ticket system               23472382  Georgia Tech   
4      Customer ticket system               92348239           UNC   

                                   Email Sent  
0                 harishthangu@gmail.com  Yes  
1  harish.thangaraj2021@vitstudent.ac.in  Yes  
2                aiis